In [1]:
import numpy as np
import pandas as pd
from moneyed import Money
from moneyed.localization import format_money
import scipy.optimize as op
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [30]:
def convtomoney(arr):
    """ Converts array of floats/integers into an array of formatted money objects. """
    floatarr = np.array(arr)
    flat = floatarr.flatten()
    nelem = len(flat)
    dim = floatarr.shape
    monarr = np.zeros(nelem, dtype = object)
    for i in range(nelem):
        monarr[i] = format_money(Money(amount = str(flat[i]), currency = 'USD'), locale = 'en_US')
    return monarr.reshape(dim)

def backschedule(Ptot0, tau0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball):

    Ptot, I, Di, Df = np.zeros([4, nper])
    r0 = intrate(tau0, tsh, rsh, tlong, rlong, spread, peryr)
    Ptot[-1] = np.around((1 + rinc)**(np.floor(nper/incper) - 1)*np.around(Ptot0,2),2)
    tf = np.around(tfees/peryr, 2)*np.ones(nper)
    Df[-1] = Dball
    Di[-1] = np.around((Ptot[-1] + Dball - tf[0])/(1 + r0/peryr),2)
    I[-1] = np.around(Di[-1]*r0/peryr, 2)
    for i in np.flipud(range(nper - 1)):
        if (i+1)%incper == 0:
            Ptot[i] = np.around(Ptot[i+1]/(1+rinc), 2)
        else:
            Ptot[i] = Ptot[i+1]
        Df[i] = Di[i+1]
        Di[i] = np.around((Df[i] + Ptot[i] - tf[i])/(1 + r0/peryr), 2)
        I[i] = np.around(Di[i]*r0/peryr, 2)
    Pp = Ptot - I - tf
    periods = np.linspace(1, nper, nper).astype(int)
    tauarr = Pp*periods/peryr
    D0 = Di[0]
    tauav = np.sum(tauarr)/(D0*peryr)
    values = np.array([Di, I, Pp, tf, Ptot, Df, tauarr])
    tab = pd.DataFrame(convtomoney(values.T), index = periods, columns = ['Initial balance', 'Interest paid', 'Principal paid', 'Trustee fees',
                                                                              'Total paid', 'Final Balance', 'Pp*period'])
    tab.columns.names = ['Period']
    return tab

In [37]:
def proceeds(tau0, Ptot0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball):
    r0 = intrate(tau0, tsh, rsh, tlong, rlong, spread, peryr)
    Ptot = np.around((1 + rinc)**(np.floor(nper/incper) - 1)*np.around(Ptot0,2),2)
    tf = np.around(tfees/peryr, 2)
    Df = Dball
    Di = np.around((Ptot + Dball - tf)/(1 + r0/peryr),2)
    I = np.around(Di*r0/peryr, 2)
    tauav = nper*(Ptot - I)
    for i in np.flipud(range(nper - 1)):
        if (i+1)%incper == 0:
            Ptot = np.around(Ptot/(1+rinc), 2)
        Df = Di
        Di = np.around((Df + Ptot - tf)/(1 + r0/peryr), 2)
        I = np.around(Di*r0/peryr, 2)
        Pp = Ptot - I - tf
        tauav = tauav + Pp*(i + 1)
    Di = np.around(Di, 2)
    tauav = tauav/(Di*peryr)
    return np.array([Di, tauav - tau0])

def findtau(Ptot0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball):

    def f1(tau0):
        r = proceeds(tau0, Ptot0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)[1]
        return r
    
    root = op.brentq(f1, 0.25*nper/peryr, nper/peryr)
    
    return root

def findPtot0(D0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball):
    
    def f1(Ptot0):
        Ptot0 = np.around(Ptot0, 2)
        tau0 = findtau(Ptot0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)
        # print("Tau0 = " + str(tau0))
        r = proceeds(tau0, Ptot0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)[0] - D0
        return r
    
    r00 = intrate(0.25*nper/peryr, tsh, rsh, tlong, rlong, spread, peryr)
    r01 = intrate(nper/peryr, tsh, rsh, tlong, rlong, spread, peryr)
    a = -np.pmt(r00/peryr, nper, D0, fv = Dball)/2.
    b = -np.pmt(r01/peryr, nper, D0, fv = Dball)*2
    root = op.brentq(f1, a, b)
    return root

In [3]:
Ptot0 = 700000
tfees = 5e3
D0 = 165600e3
tau0 = 10
tsh = 10*12
rsh = 1.96/100.
tlong = 30*12
rlong = 3.12/100.
rinc = 1.5/100.
incper = 12.
nper = 100
peryr = 12.
spread = 2/100.
Dball = 0

In [35]:
def buildschedule(D0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball):
    Ptot0 = findPtot0(D0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)
    tau0 = findtau(Ptot0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)
    r0 = intrate(tau0, tsh, rsh, tlong, rlong, spread, peryr)*100
    print["Average life = " + str(np.around(tau0, 2)) + ' years', "Interest rate = " + str(r0) + '%']
    return backschedule(Ptot0, tau0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)

In [38]:
def intrate(tau0, tsh, rsh, tlong, rlong, spread, peryr):
    r0 = (tau0*peryr - tsh)*(rlong - rsh)/(tlong - tsh) + rsh + spread
    return np.around(peryr*(((1. + r0/2.)**(2./peryr))-1.), 4)

In [39]:
intrate(4.52, tsh, rsh, tlong, rlong, spread, peryr)

0.0361

In [40]:
buildschedule(D0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)

['Average life = 4.52 years', 'Interest rate = 3.61%']


Period,Initial balance,Interest paid,Principal paid,Trustee fees,Total paid,Final Balance,Pp*period
1,"$165,600,000.26","$498,180.00","$1,323,908.11",$416.67,"$1,822,504.78","$164,276,092.15","$110,325.68"
2,"$164,276,092.15","$494,197.24","$1,327,890.87",$416.67,"$1,822,504.78","$162,948,201.28","$221,315.15"
3,"$162,948,201.28","$490,202.51","$1,331,885.60",$416.67,"$1,822,504.78","$161,616,315.68","$332,971.40"
4,"$161,616,315.68","$486,195.75","$1,335,892.36",$416.67,"$1,822,504.78","$160,280,423.32","$445,297.45"
5,"$160,280,423.32","$482,176.94","$1,339,911.17",$416.67,"$1,822,504.78","$158,940,512.15","$558,296.32"
6,"$158,940,512.15","$478,146.04","$1,343,942.07",$416.67,"$1,822,504.78","$157,596,570.08","$671,971.04"
7,"$157,596,570.08","$474,103.01","$1,347,985.10",$416.67,"$1,822,504.78","$156,248,584.98","$786,324.64"
8,"$156,248,584.98","$470,047.83","$1,352,040.28",$416.67,"$1,822,504.78","$154,896,544.70","$901,360.19"
9,"$154,896,544.70","$465,980.44","$1,356,107.67",$416.67,"$1,822,504.78","$153,540,437.03","$1,017,080.75"
10,"$153,540,437.03","$461,900.81","$1,360,187.30",$416.67,"$1,822,504.78","$152,180,249.73","$1,133,489.42"


In [124]:
Dvec

array([ 9242815.72,  9270790.55,  9298847.52,  9326924.54,  9354646.68,
        9382651.4 ,  9410707.07,  9438541.75,  9466607.98,  9494540.18,
        9522473.98,  9550436.41,  9578556.81,  9606327.19,  9634264.06,
        9662153.64,  9690384.94,  9718214.43,  9746330.04,  9774211.87,
        9802038.24,  9829952.9 ,  9857961.4 ,  9885981.33,  9914143.75,
        9941801.24,  9970000.44,  9997847.23, 10025630.13, 10053991.13,
       10081754.48, 10109710.73, 10137584.41, 10165488.13, 10193673.11,
       10221288.71, 10249545.85, 10277643.39, 10305508.81, 10333374.3 ,
       10361450.57, 10389248.9 , 10417336.59, 10445098.73, 10473242.57,
       10501109.39, 10529182.27, 10557148.16, 10585066.4 , 10613018.8 ,
       10641002.65, 10668902.19, 10697209.6 , 10724860.15, 10752571.34,
       10780735.72, 10808837.14, 10836850.3 , 10864507.27, 10892670.22,
       10920687.59, 10948529.34, 10976451.92, 11004361.64, 11032449.11,
       11060155.66, 11088355.21, 11116437.04, 11144364.16, 11172

In [106]:
tauvec = np.linspace(0.1, 50, 1000)
diffvec = np.zeros(len(tauvec))
for i in range(len(tauvec)):
    diffvec[i] = proceeds(tauvec[i], Ptot0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, Dball)[1]

In [116]:
maxpro(P0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, 0)

[130290921.38, 12.107518837446927]

In [93]:
backschedule(P0, tau0, tsh, rsh, tlong, rlong, spread, tfees, rinc, incper, nper, peryr, 0)

11.855698020666221


Period,Initial balance,Interest paid,Principal paid,Trustee fees,Total paid,Final Balance,Pp*period
1,"$175,964,596.17","$498,566.00","$391,434.00",$417.00,"$890,000.00","$175,573,579.53","$391,434.00"
2,"$175,573,579.53","$497,458.00","$392,542.00",$417.00,"$890,000.00","$175,181,455.01","$785,084.00"
3,"$175,181,455.01","$496,347.00","$393,653.00",$417.00,"$890,000.00","$174,788,219.47","$1,180,959.00"
4,"$174,788,219.47","$495,233.00","$394,767.00",$417.00,"$890,000.00","$174,393,869.76","$1,579,068.00"
5,"$174,393,869.76","$494,116.00","$395,884.00",$417.00,"$890,000.00","$173,998,402.72","$1,979,420.00"
6,"$173,998,402.72","$492,995.00","$397,005.00",$417.00,"$890,000.00","$173,601,815.19","$2,382,030.00"
7,"$173,601,815.19","$491,872.00","$398,128.00",$417.00,"$890,000.00","$173,204,104.00","$2,786,896.00"
8,"$173,204,104.00","$490,745.00","$399,255.00",$417.00,"$890,000.00","$172,805,265.96","$3,194,040.00"
9,"$172,805,265.96","$489,615.00","$400,385.00",$417.00,"$890,000.00","$172,405,297.88","$3,603,465.00"
10,"$172,405,297.88","$488,482.00","$401,518.00",$417.00,"$890,000.00","$172,004,196.56","$4,015,180.00"


0.040818
0.0405
12.105795225106812


Period,Initial balance,Interest paid,Principal paid,Trustee fees,Total paid,Final Balance,Pp*period
1,"$165,599,973.02","$558,900.00","$330,703.00",$417.00,"$889,603.00","$165,269,686.93","$330,703.00"
2,"$165,269,686.93","$557,785.00","$331,818.00",$417.00,"$889,603.00","$164,938,286.12","$663,636.00"
3,"$164,938,286.12","$556,667.00","$332,936.00",$417.00,"$889,603.00","$164,605,766.84","$998,808.00"
4,"$164,605,766.84","$555,544.00","$334,059.00",$417.00,"$889,603.00","$164,272,125.30","$1,336,236.00"
5,"$164,272,125.30","$554,418.00","$335,185.00",$417.00,"$889,603.00","$163,937,357.72","$1,675,925.00"
6,"$163,937,357.72","$553,289.00","$336,314.00",$417.00,"$889,603.00","$163,601,460.30","$2,017,884.00"
7,"$163,601,460.30","$552,155.00","$337,448.00",$417.00,"$889,603.00","$163,264,429.23","$2,362,136.00"
8,"$163,264,429.23","$551,017.00","$338,586.00",$417.00,"$889,603.00","$162,926,260.68","$2,708,688.00"
9,"$162,926,260.68","$549,876.00","$339,727.00",$417.00,"$889,603.00","$162,586,950.81","$3,057,543.00"
10,"$162,586,950.81","$548,731.00","$340,872.00",$417.00,"$889,603.00","$162,246,495.77","$3,408,720.00"


In [5]:
(1180458.04 - 417.)/(1 + 0.0405/12.)

1176071.7976828206

In [6]:
r0 = 4.08/100
np.around(0.0005*(np.around(r0/0.0005) - 1), 6)

0.0405

In [169]:
import tkinter

ImportError: No module named tkinter